In [ ]:
!pip install kfp
!pip install pandas
!pip install scikit-learn

In [2]:
from typing import List

from kfp import client
from kfp import compiler
from kfp import dsl
from kfp.dsl import Dataset
from kfp.dsl import Input
from kfp.dsl import Model
from kfp.dsl import Output
from kfp.dsl import ClassificationMetrics

In [11]:
import pandas as pd

csv_url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data'
col_names = [
    'Sepal_Length', 'Sepal_Width', 'Petal_Length', 'Petal_Width', 'Labels'
]
df = pd.read_csv(csv_url, names=col_names)

with open('dataset.csv', 'w') as f:
    df.to_csv(f)

In [12]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler

with open('dataset.csv') as f:
    df = pd.read_csv(f)
labels = df.pop('Labels')

scaler = MinMaxScaler()

df = pd.DataFrame(scaler.fit_transform(df))
df['Labels'] = labels
with open('normalized-dataset.csv', 'w') as f:
    df.to_csv(f)

In [18]:
import pickle

import pandas as pd
from sklearn.neighbors import KNeighborsClassifier

from sklearn.metrics import roc_curve
from sklearn.model_selection import train_test_split, cross_val_predict
from sklearn.metrics import confusion_matrix


with open('normalized-dataset.csv') as f:
    df = pd.read_csv(f)

y = df.pop('Labels')
X = df

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

clf = KNeighborsClassifier(n_neighbors=3)
clf.fit(X_train, y_train)


predictions = cross_val_predict(
    clf, X_train, y_train, cv=3)

y_scores = cross_val_predict(
    clf, X_train, y_train, cv=3, method='predict_proba')
y_predict = cross_val_predict(clf, X_train, y_train, cv=3, method='predict')
fpr, tpr, thresholds = roc_curve(
    y_true=y_train, y_score=y_scores[:, 1], pos_label=True)

/home/rmartine/Development/github-projects/data-science-pipelines-examples/.venv/lib64/python3.11/site-packages/sklearn/metrics/_ranking.py:1133: UndefinedMetricWarning: No positive samples in y_true, true positive value should be meaningless
  warnings.warn(


In [33]:
import math
import numpy as np

#thresholds[0] == np.nan
#thresholds[0] == np.inf